## Data Collection (S&P 500, DAX)

Collect data from 04.01.2000 to 29.11.2024 from $\href{https://finance.yahoo.com/}{\text{Yahoo Finance}}$.


### S&P 500 data collection

In [133]:
import yfinance as yf
import pandas as pd
import numpy as np

# Get list of tickers for companies from S&P500
tickers_sp500_df = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

tickers_sp500 = tickers_sp500_df.Symbol.to_list()
print(f"Number of current S&p500 tikers: {len(tickers_sp500)}")

Number of current S&p500 tikers: 503


In [194]:
# Get table with changes in list of S&P 500 companies starting from 01/01/2000
changes_sp500_df = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[1][:348]

# List of companies added to the list from 01/01/2000 to 29/11/2024
sp500_added = set(changes_sp500_df[('Added','Ticker')].to_list())

# List of companies removed from the list from 01/01/2000 to 29/11/2024
sp500_removed = set(changes_sp500_df[('Removed','Ticker')].to_list())

print(f"""Number of added companies from 01/01/2000 to 29/11/2024: {len(sp500_added)},
Number of removed companies from 01/01/2000 to 29/11/2024: {len(sp500_removed)}""")

# Select companies that were included in the index from 01/01/2000 to 29/11/2024
tickers_sp500_changes = sp500_added | sp500_removed
result_sp500 = set(tickers_sp500) - tickers_sp500_changes
print(f"Number of S&p500 companies that were in the index from 01/01/2000 to 29/11/2024: {len(result_sp500)}")

Number of added companies from 01/01/2000 to 29/11/2024: 333,
Number of removed companies from 01/01/2000 to 29/11/2024: 323
Number of S&p500 companies that were in the index from 01/01/2000 to 29/11/2024: 284


In [102]:
# Get the data for this tickers from yahoo finance
data_sp500 = yf.download(result_sp500,
                   start="2000-01-01",
                   end="2025-01-01",
                   interval='1d'
                   )['Close'] # or 'Adj Close'

[*********************100%***********************]  284 of 284 completed
ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
ERROR:yfinance:['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2000-01-01 -> 2025-01-01)')


Filter missing data. Exclude companies containing "NaN'.

In [123]:
df = data_sp500.isna().any()
nan_cols = list(df.index[df == True])
# nan_cols.remove('BF.B')
# nan_cols.remove('BRK.B')
print(f"Number of columns, containing nan: {len(nan_cols)}")
df_nan = data_sp500[nan_cols]
# df_nan

Number of columns, containing nan: 32


In [206]:
# Select only columns without missing data
filtered_cols = set(data_sp500.columns) - set(nan_cols)
data_sp500_no_nan = data_sp500[list(filtered_cols)]

In [ ]:
# Reset index
data_sp500_no_nan.index = data_sp500_no_nan.index.map(lambda x: x.strftime('%Y-%m-%d'))
data_sp500_no_nan['Date'] = pd.to_datetime(data_sp500_no_nan['Date'])
data_sp500_no_nan.set_index('Date', inplace=True)

In [209]:
# Sort columns of dataset in alphabetic order
data_sp500_no_nan = data_sp500_no_nan.sort_index(axis=1)

In [210]:
data_sp500_no_nan.tail()

Ticker,A,AAPL,ABT,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,...,WEC,WELL,WFC,WM,WMB,WMT,WY,XEL,XOM,YUM
Date,,,,,,,,,,,,,,,,,,,,,
2024-11-22,133.839996,229.869995,117.760002,512.150024,214.589996,53.130001,305.149994,321.269989,93.809998,97.589996,...,100.660004,137.910004,75.959999,224.410004,59.650002,90.440002,31.240000,71.349998,121.790001,135.270004
2024-11-25,134.490005,232.869995,118.129997,518.729980,223.580002,53.009998,304.670013,319.390015,93.529999,98.099998,...,101.349998,135.750000,76.900002,225.649994,58.189999,89.500000,32.230000,71.599998,119.970001,137.050003
2024-11-26,134.369995,235.059998,117.959999,525.299988,219.050003,53.720001,307.970001,317.959991,94.269997,99.330002,...,101.550003,138.960007,77.349998,227.449997,58.369999,91.309998,31.910000,71.870003,117.970001,138.630005
2024-11-27,138.139999,234.929993,118.949997,513.679993,217.169998,54.369999,306.920013,290.640015,94.980003,99.889999,...,101.820000,139.300003,77.209999,228.460007,58.139999,91.879997,32.349998,72.919998,117.660004,139.330002
2024-11-29,137.970001,237.330002,118.769997,515.929993,218.050003,54.599998,306.929993,291.899994,94.389999,99.860001,...,101.050003,138.179993,76.169998,228.220001,58.520000,92.500000,32.259998,72.559998,117.959999,138.940002


### DAX data colletion

In [134]:
# Get list of tickers for companies from DAX
tickers_dax_df = pd.read_html(
    'https://en.wikipedia.org/wiki/DAX')[4]

tickers_dax = tickers_dax_df.Ticker.to_list()
print(f"Number of current DAX tikers: {len(tickers_dax)}")

Number of current DAX tikers: 40


In [151]:
tickers_dax_df.head()

,Logo,Company,Prime Standard Sector,Ticker,Index weighting (%)1,Employees,Founded
0,NaN,Adidas,Apparel,ADS.DE,2.0,"061,401 (2021)",1924
1,NaN,Airbus,Aerospace & Defence,AIR.PA,6.0,"126,495 (2021)",1970
2,NaN,Allianz,Financial Services,ALV.DE,7.1,"155,411 (2021)",1890
3,NaN,BASF,Chemicals,BAS.DE,3.5,"111,047 (2021)",1865
4,NaN,Bayer,Pharmaceuticals,BAYN.DE,4.8,"099,637 (2021)",1863


In [157]:
df_tickers = tickers_dax_df[['Company', 'Ticker']]

In [189]:
# Get table with changes in list of S&P 500 companies starting from 01/01/2000
changes_dax_df = pd.read_html(
    'https://en.wikipedia.org/wiki/DAX')[6][12:]

# List of companies added to the list from 01/01/2000 to 29/11/2024
dax_added = changes_dax_df['Component included'].to_list()

# List of companies removed from the list from 01/01/2000 to 29/11/2024
dax_removed = changes_dax_df['Component excluded'].to_list()

In [190]:
def get_tickers(companies_list, df_tickers):
    tickers_list = []
    for i in companies_list:
        if i in df_tickers.Company.to_list():
            ticker = df_tickers[df_tickers.Company == i]['Ticker'].to_list()[0]
            tickers_list.append(ticker)
        else:
            pass
    return tickers_list

In [191]:
tickers_added = get_tickers(dax_added, df_tickers)
tickers_removed =  get_tickers(dax_removed, df_tickers)

In [195]:
# Select companies that were included in the index from 01/01/2000 to 29/11/2024
tickers_dax_changes = set(tickers_added) | set(tickers_removed)
result_dax = set(tickers_dax) - tickers_dax_changes
print(f"Number of DAX companies that were in the index from 01/01/2000 to 29/11/2024: {len(result_dax)}")

Number of DAX companies that were in the index from 01/01/2000 to 29/11/2024: 16


In [227]:
# Get the data for this tickers from yahoo finance
# https://ranaroussi.github.io/yfinance/reference/api/yfinance.download.html#yfinance.download
data_dax = yf.download(result_dax,
                   start="2000-01-01",
                   end="2024-11-29",
                   interval='1d'
                   )['Close'] # or 'Adj Close'

[*********************100%***********************]  16 of 16 completed


Use only companies without missiing data.

In [228]:
df_dax = data_dax.isna().any()
nan_cols_dax = list(df_dax.index[df_dax == True])
print(f"Number of columns, containing nan: {len(nan_cols_dax)}")
df_nan_dax = data_dax[nan_cols_dax]

Number of columns, containing nan: 1


In [229]:
# Select only columns without missing data
filtered_cols_dax = set(data_dax.columns) - set(nan_cols_dax)
data_dax_no_nan = data_dax[list(filtered_cols_dax)]

In [ ]:
# Reset index
data_dax_no_nan.index = data_dax_no_nan.index.map(lambda x: x.strftime('%Y-%m-%d'))
data_dax_no_nan['Date'] = pd.to_datetime(data_dax_no_nan['Date'])
data_dax_no_nan.set_index('Date', inplace=True)

In [231]:
# Sort columns of dataset in alphabetic order
data_dax_no_nan = data_dax_no_nan.sort_index(axis=1)

In [232]:
data_dax_no_nan.tail()

Ticker,ADS.DE,ALV.DE,BAS.DE,BAYN.DE,BMW.DE,DBK.DE,DTE.DE,FRE.DE,HEI.DE,MBG.DE,MUV2.DE,RWE.DE,SAP.DE,SIE.DE,VOW3.DE
Date,,,,,,,,,,,,,,,
2024-11-22,214.399994,291.399994,42.580002,19.802,67.800003,15.552,29.400000,33.310001,118.800003,52.270000,485.899994,31.010000,225.899994,177.559998,81.800003
2024-11-25,221.199997,292.500000,43.009998,19.930,69.300003,15.708,29.580000,33.360001,119.599998,52.830002,490.000000,30.959999,223.500000,180.820007,81.800003
2024-11-26,222.000000,290.299988,42.299999,18.896,68.480003,15.552,29.780001,32.950001,118.449997,52.279999,491.799988,30.799999,223.199997,180.320007,80.440002
2024-11-27,220.399994,287.899994,42.134998,19.156,68.400002,15.454,29.910000,33.200001,117.800003,52.330002,489.799988,31.139999,220.149994,178.979996,80.320000
2024-11-28,220.399994,287.899994,42.134998,19.156,68.400002,15.454,29.910000,33.200001,117.800003,52.330002,489.799988,31.139999,220.149994,178.979996,80.320000


## Data Preprocessing

In [233]:
# Function for data preprocessing
def log_diff(df):
    return np.log(df) - np.log(df.shift(1))

### S&P 500

In [212]:
# Preprocessed dataset
sp500_preprocessed = log_diff(data_sp500_no_nan)[1:]

In [213]:
sp500_preprocessed.tail()

Ticker,A,AAPL,ABT,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,...,WEC,WELL,WFC,WM,WMB,WMT,WY,XEL,XOM,YUM
Date,,,,,,,,,,,,,,,,,,,,,
2024-11-22,0.013389,0.005890,0.004255,0.015169,0.002940,-0.001129,0.001902,0.018408,-0.001704,-0.005008,...,-0.002976,0.000218,0.014988,0.012917,-0.001508,0.022928,0.020699,-0.000420,-0.001149,0.007793
2024-11-25,0.004845,0.012966,0.003137,0.012766,0.041040,-0.002261,-0.001574,-0.005869,-0.002989,0.005212,...,0.006831,-0.015786,0.012299,0.005510,-0.024781,-0.010448,0.031198,0.003498,-0.015057,0.013073
2024-11-26,-0.000893,0.009360,-0.001440,0.012586,-0.020469,0.013305,0.010773,-0.004487,0.007881,0.012460,...,0.001971,0.023371,0.005835,0.007945,0.003089,0.020022,-0.009978,0.003764,-0.016811,0.011463
2024-11-27,0.027671,-0.000553,0.008358,-0.022369,-0.008620,0.012027,-0.003415,-0.089840,0.007503,0.005622,...,0.002655,0.002444,-0.001812,0.004431,-0.003948,0.006223,0.013695,0.014504,-0.002631,0.005037
2024-11-29,-0.001231,0.010164,-0.001514,0.004371,0.004044,0.004221,0.000033,0.004326,-0.006231,-0.000300,...,-0.007591,-0.008073,-0.013561,-0.001051,0.006515,0.006725,-0.002786,-0.004949,0.002546,-0.002803


In [116]:
# Save dataset in csv format
sp500_preprocessed.to_csv("sp500_close.csv")
# dax_preprocessed.to_csv("sp500_adj_close.csv")

### DAX

In [234]:
# Preprocessed dataset
dax_preprocessed = log_diff(data_dax_no_nan)[1:]

In [235]:
dax_preprocessed.tail()

Ticker,ADS.DE,ALV.DE,BAS.DE,BAYN.DE,BMW.DE,DBK.DE,DTE.DE,FRE.DE,HEI.DE,MBG.DE,MUV2.DE,RWE.DE,SAP.DE,SIE.DE,VOW3.DE
Date,,,,,,,,,,,,,,,
2024-11-22,0.015511,0.001030,0.011099,0.014446,-0.002063,-0.029024,0.015079,0.015734,0.004218,0.008068,0.003711,0.007770,0.005326,0.008825,0.006870
2024-11-25,0.031224,0.003768,0.010048,0.006443,0.021883,0.009981,0.006104,0.001500,0.006711,0.010657,0.008403,-0.001614,-0.010681,0.018194,0.000000
2024-11-26,0.003610,-0.007550,-0.016646,-0.053276,-0.011903,-0.009981,0.006739,-0.012366,-0.009662,-0.010465,0.003667,-0.005181,-0.001343,-0.002769,-0.016766
2024-11-27,-0.007233,-0.008302,-0.003908,0.013666,-0.001169,-0.006321,0.004356,0.007559,-0.005503,0.000956,-0.004075,0.010978,-0.013759,-0.007459,-0.001493
2024-11-28,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [236]:
# Save dataset in csv format
dax_preprocessed.to_csv("dax_close.csv")
# dax_preprocessed.to_csv("dax_adj_close.csv")